In [1]:
import os

# 设置文件夹路径和类别名称
folders = [r'..\data\Test_EEG\HC',
           r'..\data\Test_EEG\MDD',
           r'..\data\Test_EEG\BD']
class_names = ['HC', 'MDD', 'BD']

# 创建存储数据集的字典
data_file = {'filename': []}

# 遍历每个类别的文件夹
for folder, class_name in zip(folders, class_names):
    # 获取文件夹中的Clean.mat文件列表
    file_list = os.listdir(folder)
    file_list = [os.path.join(folder, file) for file in file_list if file.endswith('Clean.mat')]

    # 将数据加入到对应的数据集中
    data_file['filename'].extend([(file, class_name) for file in file_list])


In [2]:
import numpy as np
import scipy.io as sio

label_dic = {'HC': 0, 'MDD': 1, "BD": 2}
target_shape = (284, 16, 24000)
X = np.empty(target_shape)
y = {'label': np.empty(284, dtype=int)}

num_merged = 0

for sub, label in data_file['filename']:  # 被试循环

    data = sio.loadmat(sub)
    sample = data['EEG_ECClean']
    eeg_data = sample["data"][0][0][:, 0:24000]
    X[num_merged] = eeg_data
    y['label'][num_merged] = label_dic[label]
    num_merged += 1

In [3]:
from torcheeg import transforms
from torcheeg.datasets import NumpyDataset

dataset = NumpyDataset(X=X, y=y)
print(dataset[0])

[NUMPY]:   0%|          | 0/2 [00:00<?, ?it/s]

The current io_size is not enough, and double the LMDB map size to 20971520 automatically.
The current io_size is not enough, and double the LMDB map size to 41943040 automatically.
The current io_size is not enough, and double the LMDB map size to 83886080 automatically.
The current io_size is not enough, and double the LMDB map size to 167772160 automatically.
The current io_size is not enough, and double the LMDB map size to 335544320 automatically.
The current io_size is not enough, and double the LMDB map size to 671088640 automatically.


[NUMPY]:  50%|█████     | 1/2 [00:01<00:01,  1.05s/it]

The current io_size is not enough, and double the LMDB map size to 1342177280 automatically.


[NUMPY]: 100%|██████████| 2/2 [00:02<00:00,  1.19s/it]

Please wait for the writing process to complete...
(array([[-1.44045854, -0.43012276,  1.58365417, ..., -5.12197399,
        -4.41983366, -3.1613338 ],
       [-2.54928565, -3.83311224, -3.79554558, ..., -3.7846837 ,
        -3.47788382, -3.46887398],
       [-0.90644282, -0.02179096,  0.66771239, ..., -0.72422498,
        -1.52700377, -1.8246603 ],
       ...,
       [-0.60493821, -0.05604951,  0.75064021, ...,  1.48914242,
         1.95111287,  1.68770337],
       [ 0.40879893, -1.10817182, -1.86888313, ..., -0.81465912,
        -1.5469166 , -1.16325331],
       [ 2.43667531,  3.24103284,  2.26753902, ...,  5.27654648,
         5.41562891,  4.28534937]]), {'clip_id': '0_0', 'label': 0})


In [4]:
from torcheeg.model_selection import KFold

k_fold = KFold(n_splits=10,
                      shuffle=True,
                      random_state=42)

In [5]:
from torch import nn
import torch

class CNN(torch.nn.Module):
    def __init__(self, in_channels=16, num_classes=3):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels, 64, kernel_size=4, stride=1, padding=2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=4, stride=1, padding=2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=4, stride=1, padding=2),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.Conv1d(256, 64, kernel_size=4, stride=1, padding=2),
            nn.ReLU()
        )

        # Calculate the output size of the convolutional layers
        self.output_size = self._get_conv_output_size()

        self.lin1 = nn.Linear(self.output_size, 1024)
        self.lin2 = nn.Linear(1024, num_classes)

    def _get_conv_output_size(self):
        # Dummy input to calculate the output size of the convolutional layers
        dummy_input = torch.zeros(1, 16, 24000)
        x = self._forward_conv(dummy_input)
        return x.view(x.size(0), -1).size(1)

    def _forward_conv(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)
        x = self.lin1(x)
        x = self.lin2(x)
        return x
# class CNN(torch.nn.Module):
#     def __init__(self, in_channels=4, num_classes=3):
#         super().__init__()
#         self.conv1 = nn.Sequential(
#             nn.ZeroPad2d((1, 2, 1, 2)),
#             nn.Conv2d(in_channels, 64, kernel_size=4, stride=1),
#             nn.ReLU()
#         )
#         self.conv2 = nn.Sequential(
#             nn.ZeroPad2d((1, 2, 1, 2)),
#             nn.Conv2d(64, 128, kernel_size=4, stride=1),
#             nn.ReLU()
#         )
#         self.conv3 = nn.Sequential(
#             nn.ZeroPad2d((1, 2, 1, 2)),
#             nn.Conv2d(128, 256, kernel_size=4, stride=1),
#             nn.ReLU()
#         )
#         self.conv4 = nn.Sequential(
#             nn.ZeroPad2d((1, 2, 1, 2)),
#             nn.Conv2d(256, 64, kernel_size=4, stride=1),
#             nn.ReLU()
#         )
#
#         self.lin1 = nn.Linear(9 * 9 * 64, 1024)
#         self.lin2 = nn.Linear(1024, num_classes)
#
#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.conv2(x)
#         x = self.conv3(x)
#         x = self.conv4(x)
#
#         x = x.flatten(start_dim=1)
#         x = self.lin1(x)
#         x = self.lin2(x)
#         return x

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn = nn.CrossEntropyLoss()
batch_size = 32

In [7]:
# from torch.utils.data import DataLoader
#
# for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#     for batch_idx, batch in enumerate(train_loader):
#         X = batch[0]
#         y = batch[1]['label']

In [8]:
# X.shape

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch in enumerate(dataloader):
        X = batch[0].to(device)
        y = batch[1]['label'].to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def valid(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X = batch[0].to(device)
            y = batch[1].to(device)

            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")

In [10]:
from torch.utils.data import DataLoader

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):

    model = CNN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    epochs = 50
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_loader, model, loss_fn, optimizer)
        valid(val_loader, model, loss_fn)
    print("Done!")

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.86 GiB (GPU 0; 4.00 GiB total capacity; 914.50 KiB already allocated; 3.30 GiB free; 2.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF